<a href="https://colab.research.google.com/github/Swarnlataaa/GEN_AI/blob/main/GAN_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Define hyperparameters
random_dim = 100
learning_rate = 0.0002
batch_size = 128
epochs = 10000


In [4]:
# Load the MNIST dataset
(train_images, _), (_, _) = mnist.load_data()
train_images = train_images / 127.5 - 1.0  # Normalize to the range [-1, 1]
train_images = np.expand_dims(train_images, axis=-1)

In [5]:
# Build the generator network
def build_generator():
    model = tf.keras.Sequential()
    model.add(Dense(256, input_dim=random_dim, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(784, activation='tanh'))
    model.add(Reshape((28, 28, 1)))
    return model

In [6]:
# Build the discriminator network
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [7]:
# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate))

In [8]:
# Build and compile the generator
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate))


In [9]:
# Build and compile the GAN model
discriminator.trainable = False  # Freeze the discriminator during GAN training

gan_input = tf.keras.Input(shape=(random_dim,))
x = generator(gan_input)
gan_output = discriminator(x)

gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate))

In [10]:
# Training loop
for epoch in range(epochs):
    # Generate random noise samples for the generator
    noise = np.random.normal(0, 1, size=[batch_size, random_dim])
    generated_images = generator.predict(noise)

    # Create a batch of real images from the training dataset
    image_batch = train_images[np.random.randint(0, train_images.shape[0], size=batch_size)]

    # Create labels for the real and generated images
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator
    discriminator_loss_real = discriminator.train_on_batch(image_batch, real_labels)
    discriminator_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)


Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 11ms/step


In [13]:
# Train the generator (via the GAN model)
noise = np.random.normal(0, 1, size=[batch_size, random_dim])
generator_loss = gan.train_on_batch(noise, real_labels)

if (epoch + 1) % 1000 == 0:
    print(f"Epoch {epoch + 1}/{epochs}, D Loss: {discriminator_loss}, G Loss: {generator_loss}")

    # Save generated images
    if (epoch + 1) % 1000 == 0:
        generated_images = generated_images * 0.5 + 0.5  # Rescale to [0, 1]
        for i in range(10):
            plt.imshow(generated_images[i].reshape(28, 28), cmap='gray')
            plt.axis('off')
            plt.savefig(f'gan_generated_image_epoch_{epoch + 1}_sample_{i}.png')
            plt.close()


Epoch 10000/10000, D Loss: 2.8234720117481515e-10, G Loss: 24.66095733642578


In [14]:
# Save the generator model
generator.save('gan_generator_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
